In [1]:
# Install the repo as an editable package (ensures local 'src' is importable in notebooks)
%pip install -e .

import json, os, sys
repo_root = r"D:\University of San Diego\Natural Language processing and GenAI\FinAgentX Multi Agent AI System\FinAgentX-Multi-Agent-AI-System"

# Ensure the repository root is on sys.path so Python can find the local 'src' package
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)

inpath = os.path.join(repo_root,"data","sample","news_AAPL.json")
with open(inpath,"r",encoding="utf-8") as f:
    articles = json.load(f)
# Attempt to import a runner function from src.orchestrator, with sensible fallbacks
try:
    from src.orchestrator import run_articles
except Exception as _import_err:
    import importlib, inspect
    orchestrator = importlib.import_module("src.orchestrator")

    # 1) Try several common candidate function names that might be present
    for candidate in ("run_articles", "process_articles", "orchestrate", "run", "main"):
        if hasattr(orchestrator, candidate) and callable(getattr(orchestrator, candidate)):
            run_articles = getattr(orchestrator, candidate)
            break
    else:
        found = False

        # 2) Look for a class with an appropriate method (e.g., Orchestrator.run_articles)
        for _, member in inspect.getmembers(orchestrator, inspect.isclass):
            for method_name in ("run_articles", "process_articles", "orchestrate", "run", "main"):
                if hasattr(member, method_name):
                    try:
                        # try to instantiate without args; if it fails, skip
                        instance = member()
                        run_articles = getattr(instance, method_name)
                        found = True
                        break
                    except Exception:
                        continue
            if found:
                break

        # 3) As a looser fallback, pick any callable attribute with 'run' or 'article' in its name
        if not found:
            for name, member in inspect.getmembers(orchestrator):
                if callable(member) and ("run" in name.lower() or "article" in name.lower()):
                    run_articles = member
                    found = True
                    break

        # 4) Final fallback: provide a safe no-op function so the notebook can continue
        if not found:
            def run_articles(arts):
                # Return the input unchanged and print a helpful warning for debugging
                print(
                    "Warning: falling back to dummy run_articles — no suitable runner found in "
                    "src.orchestrator. Returning input articles unchanged."
                )
                return arts

# Run and normalize results to a list if necessary
results = run_articles(articles)
if not isinstance(results, list):
    try:
        results = list(results)
    except Exception:
        # leave as-is if it can't be converted
        pass

# Safely print the first summary if available
first_item = results[0] if results else None

def _extract_summary(item):
    # If item is a dict, try common summary keys
    if isinstance(item, dict):
        for key in ("summary_short", "summary", "summary_text", "summary_long", "text"):
            val = item.get(key)
            if val:
                return val
        # Fallback to string representation of dict
        return str(item)
    # If item is a list/tuple, try the first element (recursively)
    if isinstance(item, (list, tuple)) and item:
        return _extract_summary(item[0])
    # For other types, just return its string representation
    return str(item)

print("First result summary:", _extract_summary(first_item))


Obtaining file:///D:/University%20of%20San%20Diego/Natural%20Language%20processing%20and%20GenAI/FinAgentX%20Multi%20Agent%20AI%20System/FinAgentX-Multi-Agent-AI-System/notebooks
Note: you may need to restart the kernel to use updated packages.
Saved end-to-end results to D:\University of San Diego\Natural Language processing and GenAI\FinAgentX Multi Agent AI System\FinAgentX-Multi-Agent-AI-System\data\sample\out\analysis_end2end.json
First result summary: eMudhra: Should you exit stock after decent listing or hold for long term? — Key figures not found. Sentiment: neutral.


ERROR: file:///D:/University%20of%20San%20Diego/Natural%20Language%20processing%20and%20GenAI/FinAgentX%20Multi%20Agent%20AI%20System/FinAgentX-Multi-Agent-AI-System/notebooks does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.
